In [12]:
import time
import numpy as np
import sys
sys.path.append("/Users/jameskitchens/Documents/GitHub/terracotta")
import terracotta as tct
import pandas as pd
from scipy import linalg
import tskit
from glob import glob



demes = pd.read_csv(f"demes_elev_two_type.tsv", sep="\t")
demes["type"] = 0 #ignoring elevation type
samples = pd.read_csv(f"samples_elev_two_type.tsv", sep="\t")
world_map = tct.WorldMap(demes, samples)
#trees = [tct.nx_bin_ts(tskit.load(ts).simplify(), [0, 10, 100, 1000, 10000, 100000, 1000000, 10000000]).first() for ts in glob(f"trees/*")[:1]]
trees = [tskit.load(ts).simplify().first() for ts in glob(f"trees/*")[:1]]

total_number_of_edges = 0
for tree in trees:
    total_number_of_edges += tree.num_edges+1
branch_lengths = np.zeros(total_number_of_edges, dtype="int64")
edge_counter = 0
for tree in trees:
    for node in tree.nodes(order="timeasc"):
        branch_lengths[edge_counter] = int(tree.branch_length(node))
        edge_counter += 1
branch_lengths = np.unique(np.array(branch_lengths))
branch_lengths = branch_lengths[branch_lengths != 0]

transition_matrix = world_map.build_transition_matrix(migration_rates={0:2e-6})

start = time.time()
trans, log = tct.precalculate_transitions(
    branch_lengths=branch_lengths,
    transition_matrix=transition_matrix
)
print(time.time() - start)

start = time.time()
trans_slow, log_slow = tct.precalculate_transitions(
    branch_lengths=branch_lengths,
    transition_matrix=transition_matrix,
    fast=False
)
print(time.time() - start)

27.856319904327393
72.49621105194092


In [19]:
print(np.sum(percent_error >= 1e-1))

0


In [21]:
print(trans[np.where(percent_error > 0.1)])
print(trans_slow[np.where(percent_error > 0.1)])

[2.5e-323]
[5.e-324]


In [13]:
percent_error = np.divide(trans-trans_slow, trans_slow)

In [18]:
print(np.where(percent_error > 1e-14))

(array([ 89,  92,  93, ..., 358, 358, 358], shape=(767626,)), array([345, 345, 148, ..., 887, 887, 887], shape=(767626,)), array([345, 345, 148, ..., 888, 890, 891], shape=(767626,)))
[9.95412671e-01 9.95223738e-01 9.95213796e-01 ... 9.23034903e-08
 6.81959702e-09 5.06007192e-15]


In [37]:
np.where(percent_error > 0.1)

(array([3]), array([283]), array([103]))

In [46]:
log[3]

array([[-1.19998800e-04, -2.91404825e+02, -2.36128331e+02, ...,
        -1.45393767e+02, -2.10765803e+02, -9.60642772e+01],
       [-2.91404825e+02, -5.99994000e-05, -4.01838175e+02, ...,
        -1.96281195e+02, -7.14979124e+01, -3.28996304e+02],
       [-2.36128331e+02, -4.01838175e+02, -1.19998800e-04, ...,
        -3.57815923e+02, -3.60930534e+02, -1.30720842e+02],
       ...,
       [-1.45393767e+02, -1.96281195e+02, -3.57815923e+02, ...,
        -1.19998800e-04, -1.17955166e+02, -2.13009529e+02],
       [-2.10765803e+02, -7.14979124e+01, -3.60930534e+02, ...,
        -1.17955166e+02, -1.19998800e-04, -2.48462223e+02],
       [-9.60642772e+01, -3.28996304e+02, -1.30720842e+02, ...,
        -2.13009529e+02, -2.48462223e+02, -1.19998800e-04]],
      shape=(893, 893))

In [44]:
log_slow[3]

array([[-1.19998800e-04, -2.91404825e+02, -2.36128331e+02, ...,
        -1.45393767e+02, -2.10765803e+02, -9.60642772e+01],
       [-2.91404825e+02, -5.99994000e-05, -4.01838175e+02, ...,
        -1.96281195e+02, -7.14979124e+01, -3.28996304e+02],
       [-2.36128331e+02, -4.01838175e+02, -1.19998800e-04, ...,
        -3.57815923e+02, -3.60930534e+02, -1.30720842e+02],
       ...,
       [-1.45393767e+02, -1.96281195e+02, -3.57815923e+02, ...,
        -1.19998800e-04, -1.17955166e+02, -2.13009529e+02],
       [-2.10765803e+02, -7.14979124e+01, -3.60930534e+02, ...,
        -1.17955166e+02, -1.19998800e-04, -2.48462223e+02],
       [-9.60642772e+01, -3.28996304e+02, -1.30720842e+02, ...,
        -2.13009529e+02, -2.48462223e+02, -1.19998800e-04]],
      shape=(893, 893))

In [61]:
percent_error = (trans[3] - trans_slow[3]) / trans_slow[3]
print(trans[3,283,103])
print(trans_slow[3,283,103])
print(percent_error[283, 103])

2.5e-323
5e-324
4.0


In [55]:
np.divide(trans[3] - trans_slow[3], trans_slow[3])

array([[ 0.00000000e+00,  0.00000000e+00, -3.51133851e-16, ...,
        -3.75735222e-16, -2.33209823e-16,  0.00000000e+00],
       [ 2.94679295e-16,  1.11028964e-16, -3.68277245e-16, ...,
        -2.50490636e-16,  1.23159943e-16,  0.00000000e+00],
       [ 1.75566925e-16, -1.84138622e-16,  0.00000000e+00, ...,
        -2.58489680e-16, -3.63899689e-16, -3.34303775e-16],
       ...,
       [-1.87867611e-16, -1.25245318e-16, -2.58489680e-16, ...,
        -1.11035626e-16,  0.00000000e+00, -1.37424573e-16],
       [ 2.33209823e-16, -1.23159943e-16,  1.81949845e-16, ...,
         0.00000000e+00, -1.11035626e-16, -1.52210368e-16],
       [ 0.00000000e+00,  3.46339325e-16, -1.67151887e-16, ...,
        -1.37424573e-16,  1.52210368e-16,  0.00000000e+00]],
      shape=(893, 893))